# 【問題1】コンペティション内容の確認

- 何を学習し、何を予測するのか  
各顧客がお金を返済できる確率  
- どのようなファイルを作りKaggleに提出するか  
test setの各SK_ID_CURRに対して、TARGET変数の確率を予測し、ヘッダー付き以下のように提出する。  
SK_ID_CURR,TARGET  
100001,0.1  
100005,0.9  
100013,0.2  
etc.  
- 提出されたものはどういった指標値で評価されるのか  
area under the ROC curve

# 【問題2】学習と検証

データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。  
検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
#データの読み込み
df = pd.read_csv('application_train.csv')

In [3]:
#データの半数以上が欠損値の場合、特徴量を削除
drop_df = df.dropna(thresh=df.shape[0]*0.5, axis=1)

#欠損値を含む行を削除
drop_df_dropna = drop_df.dropna(axis=0)

In [4]:
corr_mat1 = drop_df_dropna.corr(method='pearson')
abs_corr_mat_SalePrice = np.fabs(corr_mat1['TARGET']).drop('TARGET')
abs_corr_mat_SalePrice.nlargest(10)

EXT_SOURCE_3                   0.169535
EXT_SOURCE_2                   0.152174
DAYS_EMPLOYED                  0.067902
DAYS_BIRTH                     0.061618
REGION_RATING_CLIENT_W_CITY    0.050473
REGION_RATING_CLIENT           0.048078
FLOORSMAX_AVG                  0.046205
FLOORSMAX_MEDI                 0.046030
DAYS_LAST_PHONE_CHANGE         0.045702
FLOORSMAX_MODE                 0.045603
Name: TARGET, dtype: float64

In [5]:
#特徴量と正解ラベルを目的関数を分割
df_train = drop_df_dropna.drop('TARGET', axis=1)
df_test_valid = drop_df_dropna['TARGET']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_test_valid, test_size=0.25, random_state=1, stratify=df_test_valid)

In [7]:
#相関係数が高かった上位3つの特徴量を抽出
X_train_top3 = X_train.loc[:, ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED']]
X_test_top3 = X_test.loc[:, ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED']]

In [8]:
#ロジスティック回帰クラスをインスタンス化させ、学習
lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train_top3, y_train)

#推定
y_pred_LR_GrLivArea = lr.predict(X_test_top3)
print(y_pred_LR_GrLivArea)

#AUCを計算
roc_auc_score(y_test, y_pred_LR_GrLivArea)

[0 0 0 ... 0 0 0]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5

# 【問題3】テストデータに対する推定

テストデータ（application_test.csv）に対して推定を行い、Kaggleに提出を行ってください。  
正しく提出が行えていれば、精度は低くても構いません。

In [9]:
#データの読み込み
df_test = pd.read_csv('application_test.csv')

In [10]:
#データの半数以上が欠損値の場合、特徴量を削除
drop_df_test = df_test.dropna(thresh=df_test.shape[0]*0.5, axis=1)

#欠損値を含む行を削除
drop_df_dropna_test = drop_df_test.dropna(axis=0)

In [11]:
#相関係数が高かった上位3つの特徴量を抽出
df_test_top3 = drop_df_dropna_test.loc[:, ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED']]

df_test_top3 = df_test_top3.fillna(df_test_top3.mean())

In [12]:
#上記の特徴量を標準化する
stdsc = StandardScaler()
stdsc.fit(df_test_top3)

X_test_std2 = stdsc.transform(df_test_top3)

In [13]:
#推定
y_pred_LR_GrLivArea2 = lr.predict(X_test_std2)
print(y_pred_LR_GrLivArea2)

[0 1 1 ... 1 1 0]


In [14]:
#推定された目的ラベルの0,1の数を表示
u, counts = np.unique(y_pred_LR_GrLivArea2, return_counts=True)
print(u)
print(counts)

[0 1]
[4891 3412]


In [15]:
#提出用のフォーマット作成
df_submission = pd.concat([df_test['SK_ID_CURR'], pd.Series(y_pred_LR_GrLivArea2, name='TARGET')], axis=1)
df_submission.to_csv('submission.csv', index=False)

# 【問題4】特徴量エンジニアリング

精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。  
- どの特徴量を使うか
- どう前処理をするか  

何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。  
精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。

- パターン1  
問題2で学習の際に使用した特徴量の多項式特徴量を作成する

In [16]:
# 多項式特徴量を作成する
poly_features = df[['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_EMPLOYED', 'TARGET']]

# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
print('Polynomial Features shape: ', poly_features.shape)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Polynomial Features shape:  (307511, 20)


In [17]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(poly_features, poly_target, test_size=0.2)

In [18]:
# ロジスティック回帰クラスをインスタンス化させ、学習
lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train1, y_train1)

# 推定
probability_proba = lr.predict_proba(X_test1)
print(probability_proba)

y_pred1 = lr.predict(X_test1)
print(y_pred1)

#AUCを計算
roc_auc_score(y_test1, y_pred1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[0.9454783  0.0545217 ]
 [0.49999963 0.50000037]
 [0.49999958 0.50000042]
 ...
 [0.49999993 0.50000007]
 [0.9454783  0.0545217 ]
 [0.49999981 0.50000019]]
[0 1 1 ... 1 0 1]


0.5347651853548843

問題2で検証を行なった結果(AUC:0.5)に比べて、予測精度が向上した(AUC:0.531)。

- パターン2  
学習に使用する特徴量を変更する  
問題2では元の特徴量に前処理を行い、その特徴量についてTARGETに関して相関係数が高い上位3つの特徴量を用いて  
学習、推定、評価を行なったが、今回は上位5つの特徴量を用いる

In [19]:
#相関係数が高かった上位5つの特徴量を抽出
X_train_top5 = X_train.loc[:, ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY']]
X_test_top5 = X_test.loc[:, ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY']]

In [20]:
#ロジスティック回帰クラスをインスタンス化させ、学習
lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train_top5, y_train)

#推定
y_pred2 = lr.predict(X_test_top5)
print(y_pred2)

#AUCを計算
print(roc_auc_score(y_test, y_pred2))

[0 0 0 ... 0 0 0]
0.5


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


問題2で学習を行う際に使用した特徴量にさらに2つの特徴量(DAYS_BIRTH, REGION_RATING_CLIENT_W_CITY)を加えたが、  
AUCの値は向上しなかった。

- パターン3  
前処理の際に、標準化を行う

In [21]:
#上記の特徴量を標準化する
stdsc = StandardScaler()
stdsc.fit(X_train_top3)

X_train_std = stdsc.transform(X_train_top3)
X_test_std = stdsc.transform(X_test_top3)

In [22]:
#ロジスティック回帰クラスをインスタンス化させ、学習
lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train_std, y_train)

#推定
y_pred3 = lr.predict(X_test_std)
print(y_pred3)

#AUCを計算
roc_auc_score(y_test, y_pred3)

[0 0 0 ... 0 0 0]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5006373486297004

問題2で学習を行う際に使用した特徴量を標準化して、学習/推定を行なった結果、  
AUDの値がわずかに向上した。

- パターン4  
新しい特徴量を作成し、その特徴量を用いて学習、推定を行う

In [23]:
df_train_domain = drop_df_dropna.copy()

df_train_domain['CREDIT_INCOME_PERCENT'] = df_train_domain['AMT_CREDIT'] / df_train_domain['AMT_INCOME_TOTAL']
df_train_domain['ANNUITY_INCOME_PERCENT'] = df_train_domain['AMT_ANNUITY'] / df_train_domain['AMT_INCOME_TOTAL']
df_train_domain['CREDIT_TERM'] = df_train_domain['AMT_ANNUITY'] / df_train_domain['AMT_CREDIT']
df_train_domain['DAYS_EMPLOYED_PERCENT'] = df_train_domain['DAYS_EMPLOYED'] / df_train_domain['DAYS_BIRTH']

In [24]:
df_train2 = df_train_domain.drop('TARGET', axis=1)
df_test_valid2 = df_train_domain['TARGET']
X_train2, X_test2, y_train2, y_test2 = train_test_split(df_train2, df_test_valid2, test_size=0.2)
X_train2 = X_train2.loc[:, ['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']]
X_test2 = X_test2.loc[:, ['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']]

In [25]:
#ロジスティック回帰クラスをインスタンス化させ、学習
lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train2, y_train2)

#推定
y_pred3 = lr.predict(X_test2)
print(y_pred3)

#AUCを計算
roc_auc_score(y_test2, y_pred3)

[0 0 0 ... 0 0 0]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5

新しい特徴量を4つ作成して学習/推定を行なったが、
問題2の結果に比べてACUの値に変化はなかった。

- パターン5  
パターン4で作成した新しい特徴量の多項式特徴量を作成する

In [26]:
# 多項式特徴量を作成する
poly_features2 = df_train_domain[['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT', 'TARGET']]

# imputer for handling missing values
imputer = Imputer(strategy = 'median')

poly_target = poly_features2['TARGET']

poly_features2 = poly_features2.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features2 = imputer.fit_transform(poly_features2)

# Create the polynomial object with specified degree
poly_transformer2 = PolynomialFeatures(degree = 3)

# Train the polynomial features
poly_transformer2.fit(poly_features2)

# Transform the features
poly_features2 = poly_transformer2.transform(poly_features2)
print('Polynomial Features shape: ', poly_features2.shape)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Polynomial Features shape:  (84575, 35)


In [27]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(poly_features2, poly_target, test_size=0.2)

In [28]:
# ロジスティック回帰クラスをインスタンス化させ、学習
lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train3, y_train3)

# 推定
probability_proba2 = lr.predict_proba(X_test3)
print(probability_proba2)

y_pred4 = lr.predict(X_test3)
print(y_pred4)

#AUCを計算
roc_auc_score(y_test3, y_pred4)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[0.92392192 0.07607808]
 [0.87895685 0.12104315]
 [0.91905944 0.08094056]
 ...
 [0.91341828 0.08658172]
 [0.9134569  0.0865431 ]
 [0.91547692 0.08452308]]
[0 0 0 ... 0 0 0]


0.5

パターン4で作成した新しい特徴量4つの多項式特徴量を作成し、学習/推定を行なったが、  
AUCの値は問題2の結果と変わらなかった。